# SMC2017: Exercise set 2

## II.1 Likelihood estimates for the stochastic volatility model

Consider the stochastic volatility model
$$
\begin{align}
x_t | x_{t - 1} &\sim N(x_t;\,\phi \cdot x_{t - 1},\,\sigma^2) \\
y_t | x_t &\sim N(y_t;\, 0,\,\beta^2 \exp(x_t))
\end{align}
$$
with parameter vector $\theta = (\phi, \sigma, \beta)$.

### a) Likelihood estimation for different values of $\beta$

Consider fixed values for $\phi = 0.98$ and $\sigma = 0.16$. $\beta$ is allowed to vary between 0 and 2.

In [ ]:
theta = [0.98, 0.16]

def likelihood_bootstrap_pf(N, y, beta=0.70):
    